**Author**: Yap Jheng Khin

**FYP II Title**: Used car dealership web application

**Purpose**:
1. This notebook explains:
    - The inputs expected by Tree SHAP explainer to support adaptive random forest regressor.
    - The validation of tree weights extraction process for adaptive random forest regressor.
    - The inputs expected by Tree SHAP explainer to support adaptive random forest classifier.
    - The validation of tree weights extraction process for adaptive random forest classifier.
2. Input: 
    - Fitted car price model (Scikit-learn random forest regressor).
    - Fitted lead scoring model (Scikit-learn random forest classifier).
    - Dictionary containing the extracted weights for the car price model.
    - Dictionary containing the extracted weights for the lead scoring model.

**Execution time**: At most 10 minute in Jupyter Notebook.

**Important**: This notebook is part of the section in *FYP2_ARF_to_Dict_Conversion.ipynb*. The section is separated into a new file since it must be run on a different Python interpreter. It is because, as of April 2021, SHAP library and River library has conflicting dependenices on Numpy library. SHAP library requires Numpy version of 1.21.5 while River library requires Numpy version of 1.22.3. The import of both libraries will fail if the respective requirements are not met.

# Setup

Ensure that the current Python interpreter path is correct. For example, if the **SHAP conda environment** is named as **arf_conda_exp_env**, the expected `sys.executable` should be C:\Users\User\miniconda3\envs\\**arf_conda_exp_env**\\python.exe.

In [1]:
import sys
print(sys.executable)

C:\Users\User\miniconda3\envs\arf_conda_exp_env\python.exe


In [2]:
# Standard libraries
import copy
import json
import os
import pickle
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SHAP library
import shap

# Scikit-learn library
from sklearn import tree as sklearn_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.validation import check_is_fitted

# User-defined libraries
from general_utils import deserialize_arf

# Regression

For initializing Tree SHAP explainer, the function internally extract the tree weights from an API-specific tree models before 
storing the information in the `TreeEnsemble` class instance. The function directly supports the weights extraction of tree models from commonly used API like Scikit-learn and XGBoost. However, the function does not directly supports the weight extraction of `AdaptiveRandomForestRegressor` from River API. Instead, the function accepts a Python dictionary that stores the tree weights which are `children_left`, `children_right`, `features`, `thresholds`, `node_sample_weight`, and `values`. Thus, the tree weights must be manually extracted into a dictionary.

To ensure the adaptive random forest regressor's tree weights are passed in the correct format, the author initializes a Tree SHAP explainer object with the random forest regressor implemented by Scikit-learn. The tree weights are then retrieved from `TreeEnsemble` class instance to understand and prove the correctness of the dictionary values that are passed into the explainer.

## Tree Weights Extraction (Scikit-learn)

Load  the fitted Scikit-learn random forest regressor.

In [3]:
with open('outputs/car_price/trf_rg.pkl', 'rb') as f:
    cp_trf_model = pickle.load(f)

Initialize the SHAP tree explainer with the random forest regressor implemented by Scikit-learn.

In [4]:
cp_tree_exp = shap.TreeExplainer(model = cp_trf_model, 
                                 feature_perturbation = 'tree_path_dependent')

Based on the <a href="https://github.com/slundberg/shap/blob/master/shap/explainers/_tree.py#L613">SHAP's source code</a>, the expected dictionary structure is shown below:

```python
{
    'internal_dtype': <?>,
    'input_dtype'   : <?>,
    'objective'     : <?>,
    'tree_output'   : <?>,
    'base_offset'   : <?>,
    'trees'         : <?>
}
```

The lookup table is retrieved from the <a href="https://github.com/slundberg/shap/blob/master/shap/explainers/_tree.py#L585">source code</a>.

In [5]:
objective_name_map = {
    "mse": "squared_error",
    "variance": "squared_error",
    "friedman_mse": "squared_error",
    "reg:linear": "squared_error",
    "reg:squarederror": "squared_error",
    "regression": "squared_error",
    "regression_l2": "squared_error",
    "mae": "absolute_error",
    "gini": "binary_crossentropy",
    "entropy": "binary_crossentropy",
    "reg:logistic": "binary_crossentropy",
    "binary:logistic": "binary_crossentropy",
    "binary_logloss": "binary_crossentropy",
    "binary": "binary_crossentropy"
}

Below are the expected values for `internal_dtype`, `input_dtype`, `objective`, `tree_output`, and `base_offset` field. 

1. The `internal_dtype` defines the data type of the tree weights.

2. The default valule of `input_dtype` is `np.float64`. If the model is implemented by Scikit-learn, the `input_dtype` value is changed to `np.float32` to get the exact matches to predictions, as documented in the <a href="https://github.com/slundberg/shap/blob/master/shap/explainers/_tree.py#L576">source code</a>.

3. The `objective` value defines which metric to explain when explaining the loss of the model. If the tree model is a classifier, then the explainer will explain binary crossentropy loss. Else if the tree model is a regressor, then the explainer will explain mean squared error loss. The `objective` value should be `squared_error` instead of `None`.

4. For Scikit-learn regressor, the tree output is set to `raw_value` since the regressor directly output the target variable when `predict` is called. 

5. The default value of `base offset` is 0 unless the regressor applies gradient boosting.

In [6]:
cp_tree_ensemble = cp_tree_exp.model

print(f'Class name that stores tree weights: {type(cp_tree_ensemble)}')
print(f'internal_dtype : {cp_tree_ensemble.internal_dtype}')
print(f'input_dtype    : {cp_tree_ensemble.input_dtype}')
print(f'objective      : {cp_tree_ensemble.objective}')
print(f'tree_output    : {cp_tree_ensemble.tree_output}')
print(f'base_offset    : {cp_tree_ensemble.base_offset}')

Class name that stores tree weights: <class 'shap.explainers._tree.TreeEnsemble'>
internal_dtype : <class 'numpy.float64'>
input_dtype    : <class 'numpy.float32'>
objective      : None
tree_output    : raw_value
base_offset    : [0.]


Below is the code that extracts the `internal_dtype`, and `objective` from the Scikit-learn object, as implemented in the <a href="https://github.com/slundberg/shap/blob/master/shap/explainers/_tree.py#L630">SHAP's source code</a>. The remaining field values are fixed and pre-determined based on the API of the model.

**Important**: Note that there is a bug since the expected value of `objective` should be `squared_error`. Based on the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html">documentation</a>, The criterion `mse` was deprecated and change to `squared_error` while the criterion `mae` was deprecated and change to `absolute_error`. However, the `objective_name_map` in the SHAP library does not reflect the latest changes. Since the criterion `squared_error` is used, the `objective` is manually assigned with the value `squared_error`, just like how `mse` should be mapped to `squared_error` based on the lookup table.

In [7]:
print(f'internal_dtype : {cp_trf_model.estimators_[0].tree_.value.dtype.type}')
print(f'criterion      : {cp_trf_model.criterion}')
print(f'objective      : {objective_name_map.get(cp_trf_model.criterion, None)}')

internal_dtype : <class 'numpy.float64'>
criterion      : squared_error
objective      : None


Below is the code that extracts all the tree weights from the Scikit-learn object. Only the first element of the `trees` is investigated.

```python
scaling = 1.0 / len(model.estimators_) # output is average of trees
self.trees = [SingleTree(e.tree_, scaling=scaling, data=data, data_missing=data_missing) \
              for e in model.estimators_]
```

Below are the expected values for `children_left`, `children_right`, `children_default`, `features`, `thresholds`, `values` and `node_sample_weight` field.

The arrays below contains the *N* array elements, where *N* is the total number of nodes in the Scikit-learn random forest regressor. The root node's index position, *i*, is 0. 

Below is the description for each field value:

1. children_left: The children_left\[*i*\] contains the index position of the left child node of the node *i*. If the node *i* is a leaf node, then the value at index position *i* is set to -1.

2. children_right: The children_right\[*i*\] contains the index position of the right child node of the node *i*. If the node *i* is a leaf node, then the value at index position *i* is set to -1.

3. features, thresholds: The split feature and split threshold for node *i*. If the node *i* is a leaf node, then the value at index position *i* is set to -2. Note that the split threshold values for nominal features are always 0.5.

4. values: The prediction value for node *i*. 

5. node_sample_weight: The weighted number of training samples reaching node *i*.

For example, features\[children_right\[0\]\] is the feature of the right child of the root node. 

It can be observed that the values of `children_default` is the same as `children_left` since the  Scikit-learn random forest regressor does not have default children.

In [8]:
first_single_tree = cp_tree_ensemble.trees[0]

MAX_NODES = 10

print(f'children_left     : {first_single_tree.children_left[:MAX_NODES+10]}')
print(f'children_default  : {first_single_tree.children_default[:MAX_NODES+10]}')
print(f'children_right    : {first_single_tree.children_right[:MAX_NODES+10]}')
print(f'features          : {first_single_tree.features[:MAX_NODES].tolist()}')
print(f'thresholds        : {first_single_tree.thresholds[:MAX_NODES].tolist()}')
values = np.around(first_single_tree.values[:MAX_NODES], 4)
print(f'values            : {values[:4].tolist()}')
print(f'node_sample_weight: {first_single_tree.node_sample_weight[:MAX_NODES].tolist()}')

children_left     : [ 1  2  3  4  5  6  7  8  9 10 -1 -1 13 -1 15 -1 17 18 -1 -1]
children_default  : [ 1  2  3  4  5  6  7  8  9 10 -1 -1 13 -1 15 -1 17 18 -1 -1]
children_right    : [250 231 222 133  88  57  24  23  12  11  -1  -1  14  -1  16  -1  20  19
  -1  -1]
features          : [3, 9, 14, 1, 8, 0, 3, 55, 4, 2]
thresholds        : [1960.0, 241.5, 0.5, 90137.0, 1558.0, 2015.5, 1495.5, 0.5, 2415.0, 3852.5]
values            : [[4673.3377], [2773.3733], [2492.4839], [2405.4913]]
node_sample_weight: [8533.0, 4479.0, 4171.0, 4015.0, 2290.0, 1587.0, 1104.0, 342.0, 313.0, 77.0]


Below is the code that extracts the `children_left`, `children_right`, `features`, `thresholds`, `values`, and `node_sample_weight` from the Scikit-learn object, as implemented in the <a href="https://github.com/slundberg/shap/blob/master/shap/explainers/_tree.py#L1137">SHAP's source code</a>. 

It can be observed that:
1. Each element in the `values` contains the prediction value, which represents the computed mean of weighted targets at node *i*. Each prediction value's shape is (1,) since the number of model output is 1. The `values` is scaled down by the total number of base learners. 

2. Each element in `n_node_samples` contains the actual count of samples at node *i*. 

3. Similar to `n_node_samples`, each element in `weighted_n_node_samples` also contains the actual count of samples at node *i*, but weighted by the sample_weight. If the bootstrap hyperparameter is set to False during fitting, the `n_node_samples` will be equivalent to `weighted_n_node_samples` since all samples are equally weighted, assuming that the sample_weight is not explicitly given.

In [9]:
decision_tree = cp_trf_model.estimators_[0].tree_

print(f'children_left     : {decision_tree.children_left[:MAX_NODES+10]}')
print(f'children_right    : {decision_tree.children_right[:MAX_NODES+10]}')
print(f'features          : {decision_tree.feature[:MAX_NODES].tolist()}')
print(f'thresholds        : {decision_tree.threshold[:MAX_NODES].tolist()}')
values = np.around(decision_tree.value[:MAX_NODES], 4)
print(f'original values   : {values[:4].tolist()}')

# Reshape to (total_number_of_nodes, number_of_outputs * max_number_of_classes)
# max_number_of_classes is always 1 for regression
values = decision_tree.value.reshape(
    decision_tree.value.shape[0], 
    decision_tree.value.shape[1] * decision_tree.value.shape[2])
# Divide by total number of base learners
scaling = 1 / len(cp_trf_model.estimators_)
values = values * scaling
values = np.around(values, 4)

print(f'converted values  : {values[:4].tolist()}')

print(f'bootstrap enabled : {cp_trf_model.bootstrap}')
print(f'node_sample       : {decision_tree.n_node_samples[:MAX_NODES].tolist()}')
print(f'node_sample_weight: {decision_tree.weighted_n_node_samples[:MAX_NODES].tolist()}')

children_left     : [ 1  2  3  4  5  6  7  8  9 10 -1 -1 13 -1 15 -1 17 18 -1 -1]
children_right    : [250 231 222 133  88  57  24  23  12  11  -1  -1  14  -1  16  -1  20  19
  -1  -1]
features          : [3, 9, 14, 1, 8, 0, 3, 55, 4, 2]
thresholds        : [1960.0, 241.5, 0.5, 90137.0, 1558.0, 2015.5, 1495.5, 0.5, 2415.0, 3852.5]
original values   : [[[70100.0662]], [[41600.6001]], [[37387.2582]], [[36082.3701]]]
converted values  : [[4673.3377], [2773.3733], [2492.4839], [2405.4913]]
bootstrap enabled : True
node_sample       : [5385, 2826, 2629, 2536, 1453, 1010, 705, 219, 199, 50]
node_sample_weight: [8533.0, 4479.0, 4171.0, 4015.0, 2290.0, 1587.0, 1104.0, 342.0, 313.0, 77.0]


## Tree Weights Extraction (River)

Given the information above, the tree weights extraction for Adaptive Random Forest Regressor is as shown below:

1. The `internal_dtype` is set to `np.float64` as the data type of the tree weights.
2. The `input_dtype` is set to `np.float64` since it is not a model implemented by Sciki-learn.
3. The `objective` is set to `squared_error` to use the mean squared error as the metric to explain model loss.
4. The `tree_output` is set to `raw_value` since the model output directly output the target variable when `predict_one` is called.
5. The `base_offset` is set to 0 since the regressor does not use gradient boosting.

In [10]:
arf_rg_dict = {
    "internal_dtype" : np.float64,
    "input_dtype"    : np.float64,
    "objective"      : 'squared_error',
    "tree_output"    : 'raw_value', 
    "base_offset"    : 0
}
print(f'internal_dtype : {arf_rg_dict["internal_dtype"]}')
print(f'input_dtype    : {arf_rg_dict["input_dtype"]}')
print(f'objective      : {arf_rg_dict["objective"]}')
print(f'tree_output    : {arf_rg_dict["tree_output"]}')
print(f'base_offset    : {arf_rg_dict["base_offset"]}')

internal_dtype : <class 'numpy.float64'>
input_dtype    : <class 'numpy.float64'>
objective      : squared_error
tree_output    : raw_value
base_offset    : 0


Unlike Scikit-learn implemented decision trees, Hoeffding trees stores tree weights in Python objects instead of Numpy array. Each Hoeffding tree object contains an attribute called `_root`. If the root node is a parent node, the left child object and right child object can be accessed by using the syntax `_root.children[0]` and `_root.children[1]`, respectively. Since the dictionary expects an array of values, the tree weights like `features` and `thresholds` must be fetched from each node object, one at a time.

Below describes the syntax to retrieve the tree weights from each node in a Hoeffding decision tree regressor.

1. children_left:  If the current `node` is a branch node (subclass of `DTBranch`), the left child object can be accessed by using the syntax `node.children[0]`. 


2. children_right: If the current `node` is a branch node (subclass of `DTBranch`), the right child object can be accessed by using the syntax `node.children[1]`. 


3. children_default: Like Scikit-learn's decision trees, Hoeffding tree does not have default children. Hence, the `children_left` is assigned to `children_default` as the substitute.


4. features: If the current `node` is a branch node and the class type is `NumericBinaryBranch`, the **numerical** split feature can be accessed by using the syntax `node.feature` Else if the current `node` is a branch node and the class type is `NominalBinaryBranch`, the **nominal** split feature can be accessed by using the syntax `node.feature`.


5. thresholds: If the current `node` is a branch node and the class type is `NumericBinaryBranch`, the **split threshold** can be accessed by using the syntax `node.threshold`. Else if the current `node` is a branch node and the class type is `NominalBinaryBranch`, the **split value** can be accessed by using the syntax `node.value`. The **split value** cannot be directly copied to the array since the logic is different. The conversion is discussed in *FYP2_ARF_to_Dict_Conversion.ipynb*.


6. values: Both parent nodes and leaf nodes contain an attribute called `stats`. The `stats` attribute references a `Var` class instance that stores the statistical information to calculate the prediction value. The prediction value is retrieved by using the syntax `node.stats.mean.get()`, as documented in the <a href="https://github.com/online-ml/river/blob/main/river/tree/nodes/htr_nodes.py#L68">source code</a>. However, it was later proved that the `node.stats.mean.get()` cannot be used to calculate accurate SHAP values when continously training on new samples. The details can be found at `FYP2_Car_Price_Explainer.ipynb`.


7. node_sample_weight: Both parent nodes and leaf nodes contain an attribute called `stats`. The `stats` attribute references a `Var` class instance that stores the statistical information to calculate the prediction value. The weighted count of samples at the current node is retrieved from the stored statistical informaton using the syntax `node.stats.mean.n`, as documented in the <a href="https://github.com/online-ml/river/blob/main/river/stats/mean.py#L11">source code</a>. Note that the `n` is an attribute in `river.stats.mean.Mean`, and the `mean` is an attribute in `river.stats.var.Var`. However, it was later proved that the `node.stats.mean.n` cannot be used to calculate accurate SHAP values when continously training on new samples. The details can be found at `FYP2_Car_Price_Explainer.ipynb`. 

### Validation using Tree SHAP Explainer

A SHAP tree explainer is initialized with the imported dictionary to check that the extraction function is successful. 

**Reminder**: The River model is not loaded here since the River library is incompatible with SHAP library. Instead, the dictionary containing the extracted tree weights is imported here in JSON format. 

Load the dictionary containing the extracted tree weights.

In [11]:
with open('outputs/car_price/arf_rg.json', 'r') as f:
    cp_arf_dict_serializable = json.load(f)

cp_arf_dict = deserialize_arf(cp_arf_dict_serializable)

Load test data to compute SHAP value and serve as background dataset for SHAP tree loss explainer. Randomly choose 300 samples from the test set to serve as the background dataset.

In [12]:
# Load data preprocessor
with open('outputs/car_price/data_preprocessor.pkl', 'rb') as f:
    cp_data_pp = pickle.load(f)

# Load car test dataset
cp_test = pd.read_csv(f'outputs/car_price/car_test_processed.csv')

# Preprocess X test
cp_X_test = cp_test.drop(columns=['price', 'model'], axis=1)
cp_X_test = cp_data_pp.preprocess(cp_X_test)
cp_y_test = cp_test['price']

print(f'X_test shape: {cp_X_test.shape}')
print(f'y_test shape: {cp_y_test.shape}')

# Choose a subset of the cp_X_test
rng = np.random.default_rng(2022)
idx_arr = rng.choice(range(len(cp_X_test)), 300)
cp_X_test_subsample = cp_X_test.iloc[idx_arr, :].copy()

X_test shape: (1094, 76)
y_test shape: (1094,)


Initialize the SHAP tree explainer using the deserialized dictionary and calculate the SHAP value. The details of the explainer is discussed in *FYP2_Car_Price_Explainer.ipynb*. In this case, the SHAP tree explainer is using `tree_path_dependent` approach.

In [13]:
# Compute SHAP value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(cp_arf_dict)}')
for _ in range(num_iter):
    cp_tree_explainer = shap.TreeExplainer(
        model = copy.deepcopy(cp_arf_dict), 
        feature_perturbation = 'tree_path_dependent'
    )
    _ = cp_tree_explainer.shap_values(cp_X_test)

end = time.time()
print(f'Average time taken to calculate SHAP value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'dict'>
Average time taken to calculate SHAP value: 2.182078218460083 seconds


The SHAP tree explainer is also initialized using Scikit-learn random forest regressor and SHAP value is calculated. This is to check that the performance of SHAP value calculation is not affected when dictionary is used. The result shows that the performance of SHAP value calculation is similar to the previous performance. In this case, the SHAP tree explainer is using `tree_path_dependent` approach.

In [14]:
# Compute SHAP value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(cp_trf_model)}')
for _ in range(num_iter):
    cp_tree_explainer_tmp = shap.TreeExplainer(
        model = cp_trf_model, 
        feature_perturbation = 'tree_path_dependent'
    )
    _ = cp_tree_explainer_tmp.shap_values(cp_X_test)

end = time.time()
print(f'Average time taken to calculate SHAP value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
Average time taken to calculate SHAP value: 2.049235725402832 seconds


Initialize the SHAP tree explainer using the deserialized dictionary and calculate the SHAP value. The details of the explainer is discussed in *FYP2_Car_Price_Explainer.ipynb*. In this case, the SHAP tree explainer is using `interventional` approach. It can be observed that SHAP tree explainer using `interventional` approach is slower than the one using `tree_path_dependent` approach.

In [15]:
# Compute SHAP value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(cp_arf_dict)}')
for _ in range(num_iter):
    cp_tree_explainer = shap.TreeExplainer(
        model = copy.deepcopy(cp_arf_dict), 
        feature_perturbation = 'interventional', 
        data = cp_X_test_subsample
    )
    _ = cp_tree_explainer.shap_values(cp_X_test)

end = time.time()
print(f'Average time taken to calculate SHAP value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'dict'>
Average time taken to calculate SHAP value: 2.697571587562561 seconds


The SHAP tree explainer is also initialized using Scikit-learn random forest regressor and SHAP value is calculated. This is to check that the performance of SHAP value calculation is not affected when dictionary is used. The result shows that the performance of SHAP value calculation is similar to the previous performance. In this case, the SHAP tree explainer is using `interventional` approach. It can be observed that SHAP tree explainer using `interventional` approach is slower than the one using `tree_path_dependent` approach.

In [16]:
# Compute SHAP value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(cp_trf_model)}')
for _ in range(num_iter):
    cp_tree_explainer_tmp = shap.TreeExplainer(
        model = cp_trf_model, 
        feature_perturbation = 'interventional', 
        data = cp_X_test_subsample
    )
    _ = cp_tree_explainer_tmp.shap_values(cp_X_test)

end = time.time()
print(f'Average time taken to calculate SHAP value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
Average time taken to calculate SHAP value: 2.9476255655288695 seconds


### Validation using Tree SHAP Loss Explainer

A SHAP tree loss explainer is initialized with the imported dictionary to check that the extraction function is successful.

Initialize the SHAP tree loss explainer using the deserialized dictionary and calculate the SHAP loss value. The details of the explainer is discussed in *FYP2_Car_Price_Explainer.ipynb*.

In [17]:
# Compute SHAP loss value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(cp_arf_dict)}')
for _ in range(num_iter):
    cp_tree_loss_explainer = shap.TreeExplainer(
        model = copy.deepcopy(cp_arf_dict), 
        feature_perturbation = 'interventional', 
        data = cp_X_test_subsample, 
        model_output = 'log_loss'
    )
    _ = cp_tree_loss_explainer.shap_values(cp_X_test, cp_y_test)

end = time.time()
print(f'Average time taken to calculate SHAP loss value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'dict'>
Average time taken to calculate SHAP loss value: 3.3043734312057493 seconds


The SHAP tree loss explainer is also initialized using Scikit-learn random forest regressor and SHAP loss value is calculated. This is to ensure that the performance of SHAP loss value calculation is not affected when dictionary is used. The result below shows that the performance of SHAP loss value calculation is similar to previous performance.

In [18]:
# Compute SHAP loss value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(cp_trf_model)}')
for _ in range(num_iter):
    cp_tree_loss_explainer_tmp = shap.TreeExplainer(
        model = cp_trf_model, 
        feature_perturbation = 'interventional', 
        data = cp_X_test_subsample,
        model_output = 'log_loss'
    )
    # The objective is manually assigned with the value squared_error
    # to solve the bug mentioned above
    cp_tree_loss_explainer_tmp.model.objective = 'squared_error'
    _ = cp_tree_loss_explainer_tmp.shap_values(cp_X_test, cp_y_test)

end = time.time()
print(f'Average time taken to calculate SHAP loss value : {(end - start)/num_iter} seconds')

Model used in initialization: <class 'sklearn.ensemble._forest.RandomForestRegressor'>
Average time taken to calculate SHAP loss value : 3.281397891044617 seconds


### Validation using Dictionary

As shown in *FYP2_Car_Price_Model_Training.ipynb*, `cp_trf_model` is the TRF model that transfer its tree structure to the ARF model where its weights are extracted to `cp_arf_dict`. In that transfer learning process, it is validated that the tree structure is exactly the same. The tree structure is the same such that for every parent node *i*, the `split feature`, `split threshold`, and `child nodes` are the same for both models. The `prediction value` and `node_sample_weight` are different since it is technically impossible to directly copy the weights from TRF model to the ARF model. For the TRF model, the prediction value and `node_sample_weight` has already pre-calculated by the splitter for each newly created node when building the tree. For the ARF model, the prediction value and `node_sample_weight` are not directly stored. Instead, a Var class instance stores the statistical information to calculate the prediction value and the `node_sample_weight`is retrieved from the statistical information.


Given the high similarity in tree structures, the `cp_trf_model`'s weights are extracted to a dictionary named `cp_trf_dict` to compare with `cp_arf_dict`. The `cp_trf_dict` is used to initialize a SHAP tree explainer to verify its correctness.

In [19]:
cp_trf_dict = {
    'internal_dtype' : cp_trf_model.estimators_[0].tree_.value.dtype.type,
    'input_dtype'    : np.float32,
    'objective'      : 'squared_error',
    'tree_output'    : 'raw_value',
    'base_offset'    : 0,
    'trees'          : []
}

for dt in cp_trf_model.estimators_:
    dt = dt.tree_
    
    singletree = {
        'children_left'     : dt.children_left,
        'children_right'    : dt.children_right,
        'children_default'  : dt.children_left,
        'features'          : dt.feature,
        'thresholds'        : dt.threshold,
        'node_sample_weight': dt.weighted_n_node_samples
    }
    
    values = dt.value.reshape(
        dt.value.shape[0], 
        dt.value.shape[1] * dt.value.shape[2]
    )
    # Divide by total number of base learners
    scaling = 1 / len(cp_trf_model.estimators_)
    values = values * scaling
        
    singletree['values'] = values
    
    cp_trf_dict['trees'].append(singletree)
    
# Compute SHAP loss value
cp_tree_loss_explainer_tmp2 = shap.TreeExplainer(
    model = copy.deepcopy(cp_trf_dict), 
    feature_perturbation = 'interventional', 
    data = cp_X_test_subsample
)
_ = cp_tree_loss_explainer_tmp2.shap_values(cp_X_test, cp_y_test)

**Test 1**: The field values `internal_dtype`, `objective`, `tree_output`, `base_offset` for both dictionaries are the same. As mentioned earlier, for the Scikit-learn-implemented tree models, the `input_dtype` value must be set to `np.float32` to get the exact matches to predictions.

In [20]:
print('Scikit-learn random forest regressor:\n')
print(f'\tinternal_dtype: {cp_trf_dict["internal_dtype"]}')
print(f'\tinput_dtype   : {cp_trf_dict["input_dtype"]}')
print(f'\tobjective     : {cp_trf_dict["objective"]}')
print(f'\ttree_output   : {cp_trf_dict["tree_output"]}')
print(f'\tbase_offset   : {cp_trf_dict["base_offset"]}')

print('\nRiver adaptive random forest regressor:\n')
print(f'\tinternal_dtype: {cp_arf_dict["internal_dtype"]}')
print(f'\tinput_dtype   : {cp_arf_dict["input_dtype"]}')
print(f'\tobjective     : {cp_arf_dict["objective"]}')
print(f'\ttree_output   : {cp_arf_dict["tree_output"]}')
print(f'\tbase_offset   : {cp_arf_dict["base_offset"]}')

Scikit-learn random forest regressor:

	internal_dtype: <class 'numpy.float64'>
	input_dtype   : <class 'numpy.float32'>
	objective     : squared_error
	tree_output   : raw_value
	base_offset   : 0

River adaptive random forest regressor:

	internal_dtype: <class 'numpy.float64'>
	input_dtype   : <class 'numpy.float64'>
	objective     : squared_error
	tree_output   : raw_value
	base_offset   : 0


**Test 2**: The position of all the nodes must be the same. It must be ensured that for every *i*th node, the split feature and split threshold must be the same between the two dictionaries. Note that the node's array index position for *i*th node is often not the same between the two dictionaries. Hence, the node index for decision tree and Hoeffding tree must be tracked separately to access the respective arrays of tree weights.

In [21]:
df = pd.DataFrame([], columns=['base_learner_no', 'dt_node_idx', 'ht_node_idx', 
                               'same_feature?', 'same_threshold?'])

node_id = -1

for base_learner_no in range(len(cp_trf_model.estimators_)):
    queue = []
    # Separately track the node index position for both dictionaries
    queue.append((0, 0))

    while len(queue) > 0:
        dt_node_idx, ht_node_idx = queue.pop(0)
        dt_node_idx = int(dt_node_idx)
        ht_node_idx = int(ht_node_idx)

        # Retrieve the features from the dictionaries
        ht_node_feature = cp_arf_dict["trees"][base_learner_no]["features"][ht_node_idx]
        dt_node_feature = cp_trf_dict["trees"][base_learner_no]["features"][dt_node_idx]

        # Retrieve the threshold from the dictionaries
        ht_node_threshold = cp_arf_dict["trees"][base_learner_no]["thresholds"][ht_node_idx]
        dt_node_threshold = cp_trf_dict["trees"][base_learner_no]["thresholds"][dt_node_idx]

        # Add records to dataframe for debugging
        node_id += 1
        df.loc[node_id, :] = [base_learner_no+1, dt_node_idx, ht_node_idx, 
                              ht_node_feature == dt_node_feature, 
                              ht_node_threshold == dt_node_threshold]

        # Retrieve the left child index position from the dictionaries
        ht_left_ch_idx = cp_arf_dict["trees"][base_learner_no]["children_left"][ht_node_idx]
        dt_left_ch_idx = cp_trf_dict["trees"][base_learner_no]["children_left"][dt_node_idx]

        # Retrieve the right child index position from the dictionaries
        ht_right_ch_idx = cp_arf_dict["trees"][base_learner_no]["children_right"][ht_node_idx]
        dt_right_ch_idx = cp_trf_dict["trees"][base_learner_no]["children_right"][dt_node_idx]

        # Only enqueue if the child node is a branch node
        if dt_left_ch_idx != -1:
            queue.append((dt_left_ch_idx, ht_left_ch_idx))
        if dt_right_ch_idx != -1:
            queue.append((dt_right_ch_idx, ht_right_ch_idx))

df.head(10)

,base_learner_no,dt_node_idx,ht_node_idx,same_feature?,same_threshold?
0,1,0,0,True,True
1,1,1,1,True,True
2,1,250,2,True,True
3,1,2,3,True,True
4,1,231,4,True,True
5,1,251,5,True,True
6,1,400,6,True,True
7,1,3,7,True,True
8,1,222,8,True,True
9,1,232,9,True,True


The number of problematic nodes should be zero as shown below.

In [22]:
problematic_nodes = df[(~df['same_feature?']) | (~df['same_threshold?'])]
print(f'Number of problematic nodes: {problematic_nodes.shape[0]}')
problematic_nodes

Number of problematic nodes: 0


,base_learner_no,dt_node_idx,ht_node_idx,same_feature?,same_threshold?


**Test 3**: If test 2 is successful, check the average weights difference between Scikit-learn RF regressor and River ARF regressor. The average weights difference for `features` and `thresholds` must be zero since the test 2 has passed. 

Based on the result below:
1. The first set of base learners have the highest average `values` difference of around 140.0. The difference is negligible and acceptable since the difference only accounts for 2.7% (140/5192) of the average `values` in the first decision tree.
2. The third set of base learners have the highest average `node_sample_weight` difference of around 7.5. The difference is negligible and acceptable since the difference only accounts for 2.3% (7.5/330) of the average `node_sample_weight` in the third decision tree.

In [23]:
weight_types = ['features', 'thresholds', 'values', 'node_sample_weight']

df = pd.DataFrame([], columns=weight_types)

for weight_type in weight_types:
    for idx in range(len(cp_trf_model.estimators_)):
        dt_weights = np.sort(cp_arf_dict["trees"][idx][weight_type].flatten())
        ht_weights = np.sort(cp_trf_dict["trees"][idx][weight_type].flatten())
        diffs = ht_weights - dt_weights
        diffs = diffs[diffs != 0]
        diffs_mean = np.abs(diffs).mean() if diffs.shape[0] > 0 else 0
        df.loc[idx, f'{weight_type}'] = diffs_mean

print('Average weights difference between Scikit-learn RF regressor and River ARF regressor:')
display(df)

print(f'Average of values at the first DT node        : '+
      f'{cp_arf_dict["trees"][0]["values"].flatten().mean()}')

print(f'Average of node_sample_weight at third DT node: '+\
      f'{cp_arf_dict["trees"][2]["node_sample_weight"].flatten().mean()}')

Average weights difference between Scikit-learn RF regressor and River ARF regressor:


,features,thresholds,values,node_sample_weight
0,0,0,138.454519,5.473684
1,0,0,87.346089,5.990566
2,0,0,120.470146,7.460674
3,0,0,86.57759,4.44582
4,0,0,78.85451,3.808442
5,0,0,76.634644,6.287037
6,0,0,101.776429,6.02349
7,0,0,85.88998,5.21118
8,0,0,84.723226,4.815934
9,0,0,80.926406,5.27673


Average of values at the first DT node        : 5192.022179869883
Average of node_sample_weight at third DT node: 330.10601719197706


# Classification

For initializing Tree SHAP explainer, the function internally extract the tree weights from an API-specific tree models before 
storing the information in the `TreeEnsemble` class instance. The function directly supports the weights extraction of tree models from commonly used API like Scikit-learn and XGBoost. However, the function does not directly supports the weight extraction of `AdaptiveRandomForestClassifier` from River API. Instead, the function accepts a Python dictionary that stores the tree weights which are `children_left`, `children_right`, `features`, `thresholds`, `node_sample_weight`, and `values`. Thus, the tree weights must be manually extracted into a dictionary.

To ensure the adaptive random forest classifier's tree weights are passed in the correct format, the author initializes a Tree SHAP explainer object with the random forest classifier implemented by Scikit-learn. The tree weights are then retrieved from `TreeEnsemble` class instance to understand and prove the correctness of the values that are passed into the explainer.

## Tree Weights Extraction (Scikit-learn)

Load  the fitted Scikit-learn random forest classifier.

In [24]:
with open('outputs/lead_scoring/trf_cf.pkl', 'rb') as f:
    ls_trf_model = pickle.load(f)

Initialize the SHAP tree explainer with the random forest classifier implemented by Scikit-learn.

In [25]:
ls_tree_exp = shap.TreeExplainer(model = ls_trf_model, 
                                 feature_perturbation = 'tree_path_dependent')

Based on the <a href="https://github.com/slundberg/shap/blob/master/shap/explainers/_tree.py#L613">SHAP's source code</a>, the expected dictionary structure is as shown below:

```python
{
    'internal_dtype': <?>,
    'input_dtype'   : <?>,
    'objective'     : <?>,
    'tree_output'   : <?>,
    'base_offset'   : <?>,
    'trees'         : <?>
}
```

Below are the expected values for `internal_dtype`, `input_dtype`, `objective`, `tree_output`, and `base_offset` field. 

1. The `internal_dtype` defines the data type of the tree weights.

2. The default valule of `input_dtype` is `np.float64`. If the model is implemented by Scikit-learn, the `input_dtype` value is changed to `np.float32` to get the exact matches to predictions, as documented in the <a href="https://github.com/slundberg/shap/blob/master/shap/explainers/_tree.py#L576">source code</a>.

3. The `objective` value defines which metric to explain when explaining the loss of the model. If the tree model is a classifier, then the explainer will explain binary crossentropy loss. Else if the tree model is a regressor, then the explainer will explain mean squared error loss. 

4. For Scikit-learn classifier, the tree output is set to `probability` since the classifier can output the prediction probabilities by calling `predict_proba`. 

5. The default value of `base offset` is 0 unless the classifier applies gradient boosting.

In [26]:
ls_tree_ensemble = ls_tree_exp.model

print(f'Class name that stores tree weights: {type(ls_tree_ensemble)}')
print(f'internal_dtype : {ls_tree_ensemble.internal_dtype}')
print(f'input_dtype    : {ls_tree_ensemble.input_dtype}')
print(f'objective      : {ls_tree_ensemble.objective}')
print(f'tree_output    : {ls_tree_ensemble.tree_output}')
print(f'base_offset    : {ls_tree_ensemble.base_offset}')

Class name that stores tree weights: <class 'shap.explainers._tree.TreeEnsemble'>
internal_dtype : <class 'numpy.float64'>
input_dtype    : <class 'numpy.float32'>
objective      : binary_crossentropy
tree_output    : probability
base_offset    : [0. 0.]


Below is the code that extracts the `internal_dtype`, and `objective` from the Scikit-learn object, as implemented in the <a href="https://github.com/slundberg/shap/blob/master/shap/explainers/_tree.py#L684">SHAP's source code</a>. The remaining field values are fixed and pre-determined based on the API of the model.

In [27]:
print(f'internal_dtype : {ls_trf_model.estimators_[0].tree_.value.dtype.type}')
print(f'objective      : {objective_name_map.get(ls_trf_model.criterion, None)}')

internal_dtype : <class 'numpy.float64'>
objective      : binary_crossentropy


Below is the code that extracts all the tree weights from the Scikit-learn object. Only the first element of the `trees` is investigated.

```python
scaling = 1.0 / len(model.estimators_) # output is average of trees
self.trees = [SingleTree(e.tree_, normalize=True, scaling=scaling, 
                         data=data, data_missing=data_missing) \
              for e in model.estimators_]
```

Below are the expected values for `children_left`, `children_right`, `children_default`, `features`, `thresholds`, `values` and `node_sample_weight` field.

The arrays below contains the *N* array elements, where *N* is the total number of nodes in the Scikit-learn random forest classifier. The root node's index position, *i*, is 0. 

Below is the description for each field value:

1. children_left: The children_left\[*i*\] contains the index position of the left child node of the node *i*. If the node *i* is a leaf node, then the value at index position *i* is set to -1.

2. children_right: The children_right\[*i*\] contains the index position of the right child node of the node *i*. If the node *i* is a leaf node, then the value at index position *i* is set to -1.

3. features, thresholds: The split feature and split threshold for node *i*. If the node *i* is a leaf node, then the value at index position *i* is set to -2. Note that the split threshold values for nominal features are always 0.5.

4. values: The prediction value for node *i*. 

5. node_sample_weight: The weighted number of training samples reaching node *i*.

For example, features\[children_right\[0\]\] is the feature of the right child of the root node. 

It can be observed that the values of `children_default` is the same as `children_left` since the  Scikit-learn random forest classifier does not have default children.

In [28]:
first_single_tree = ls_tree_ensemble.trees[0]

MAX_NODES = 10

print(f'children_left     : {first_single_tree.children_left[:MAX_NODES+10]}')
print(f'children_default  : {first_single_tree.children_default[:MAX_NODES+10]}')
print(f'children_right    : {first_single_tree.children_right[:MAX_NODES+10]}')
print(f'features          : {first_single_tree.features[:MAX_NODES].tolist()}')
print(f'thresholds        : {first_single_tree.thresholds[:MAX_NODES].tolist()}')
values = np.around(first_single_tree.values[:MAX_NODES], 4)
print(f'values            : {values[:4].tolist()}')
print(f'node_sample_weight: {first_single_tree.node_sample_weight[:MAX_NODES].tolist()}')

children_left     : [ 1  2  3  4  5 -1 -1 -1  9 -1 -1 12 13 14 -1 -1 -1 18 19 -1]
children_default  : [ 1  2  3  4  5 -1 -1 -1  9 -1 -1 12 13 14 -1 -1 -1 18 19 -1]
children_right    : [24 11  8  7  6 -1 -1 -1 10 -1 -1 17 16 15 -1 -1 -1 21 20 -1]
features          : [1, 0, 7, 8, 6, -2, -2, -2, 3, -2]
thresholds        : [563.5, 0.5, 0.5, 0.5, 0.5, -2.0, -2.0, -2.0, 0.5, -2.0]
values            : [[0.0255, 0.0245], [0.0328, 0.0172], [0.0185, 0.0315], [0.0214, 0.0286]]
node_sample_weight: [5219.0, 3237.0, 1076.0, 911.0, 906.0, 44.0, 862.0, 5.0, 165.0, 157.0]


Below is the code that extracts the `children_left`, `children_right`, `features`, `thresholds`, `values`, and `node_sample_weight` from the Scikit-learn object, as implemented in the <a href="https://github.com/slundberg/shap/blob/master/shap/explainers/_tree.py#L1137">SHAP's source code</a>. 

It can be observed that:
1. Each element in the `values` contains the prediction value, which represents the computed weights for each class at node *i*. Each prediction value's shape is (2,) since the number of model output is 1 and the number of class labels is 2. The `values` is normalized and scaled down by the total number of base learners.

2. Each element in `n_node_samples` contains the actual count of samples at node *i*. 

3. Similar to `n_node_samples`, each element in `weighted_n_node_samples` also contains the actual count of samples at node *i*, but weighted by the sample_weight. If the bootstrap hyperparameter is set to False during fitting, the `n_node_samples` will be equivalent to `weighted_n_node_samples` since all samples are equally weighted, assuming that the sample_weight is not explicitly given.

4. Each element in `weighted_n_node_samples` is equivalent to the sum of the corresponding element in `values`, regardless of the bootstrap parameter. Taking the first few elements as an example, 5219 = 2663 + 2556, 3237 = 2123 + 1114, 1076 = 399 + 677 and so on...

In [29]:
decision_tree = ls_trf_model.estimators_[0].tree_

print(f'children_left     : {decision_tree.children_left[:MAX_NODES+10]}')
print(f'children_right    : {decision_tree.children_right[:MAX_NODES+10]}')
print(f'features          : {decision_tree.feature[:MAX_NODES].tolist()}')
print(f'thresholds        : {decision_tree.threshold[:MAX_NODES].tolist()}')
values = np.around(decision_tree.value[:MAX_NODES], 4)
print(f'original values   : {values[:4].tolist()}')

# Reshape to (total_number_of_nodes, number_of_outputs * max_number_of_classes)
values = decision_tree.value.reshape(
    decision_tree.value.shape[0], 
    decision_tree.value.shape[1] * decision_tree.value.shape[2])
# Normalize the prediction value using sum for each node
values = (values.T / values.sum(1)).T
# Divide by total number of base learners
scaling = 1 / len(ls_trf_model.estimators_)
values = values * scaling
values = np.around(values, 4)

print(f'converted values  : {values[:4].tolist()}')

print(f'bootstrap enabled : {cp_trf_model.bootstrap}')
print(f'node_sample       : {decision_tree.n_node_samples[:MAX_NODES].tolist()}')
print(f'node_sample_weight: {decision_tree.weighted_n_node_samples[:MAX_NODES].tolist()}')

children_left     : [ 1  2  3  4  5 -1 -1 -1  9 -1 -1 12 13 14 -1 -1 -1 18 19 -1]
children_right    : [24 11  8  7  6 -1 -1 -1 10 -1 -1 17 16 15 -1 -1 -1 21 20 -1]
features          : [1, 0, 7, 8, 6, -2, -2, -2, 3, -2]
thresholds        : [563.5, 0.5, 0.5, 0.5, 0.5, -2.0, -2.0, -2.0, 0.5, -2.0]
original values   : [[[2663.0, 2556.0]], [[2123.0, 1114.0]], [[399.0, 677.0]], [[390.0, 521.0]]]
converted values  : [[0.0255, 0.0245], [0.0328, 0.0172], [0.0185, 0.0315], [0.0214, 0.0286]]
bootstrap enabled : True
node_sample       : [3303, 2063, 671, 567, 565, 25, 540, 2, 104, 97]
node_sample_weight: [5219.0, 3237.0, 1076.0, 911.0, 906.0, 44.0, 862.0, 5.0, 165.0, 157.0]


## Tree Weights Extraction (River)

Given the information above, the tree weights extraction for Adaptive Random Forest Classifier is as shown below:

1. The `internal_dtype` is set to `np.float64` as the data type of the tree weights.
2. The `input_dtype` is set to `np.float64` since it is not a model implemented by Sciki-learn.
3. The `objective` is set to `binary_crossentropy` to use the binary crossentropy as the metric to explain model loss.
4. The `tree_output` is set to `probability` since the model output output the prediction probabilities when `predict_proba_one` is called.
5. The `base_offset` is set to 0 since the classifier does not use gradient boosting.

In [30]:
arf_rg_dict = {
    "internal_dtype" : np.float64,
    "input_dtype"    : np.float64,
    "objective"      : 'binary_crossentropy',
    "tree_output"    : 'probability', 
    "base_offset"    : 0
}
print(f'internal_dtype : {arf_rg_dict["internal_dtype"]}')
print(f'input_dtype    : {arf_rg_dict["input_dtype"]}')
print(f'objective      : {arf_rg_dict["objective"]}')
print(f'tree_output    : {arf_rg_dict["tree_output"]}')
print(f'base_offset    : {arf_rg_dict["base_offset"]}')

internal_dtype : <class 'numpy.float64'>
input_dtype    : <class 'numpy.float64'>
objective      : binary_crossentropy
tree_output    : probability
base_offset    : 0


Unlike Scikit-learn implemented decision trees, Hoeffding trees stores tree weights in Python objects instead of Numpy array. Each Hoeffding tree object contains an attribute called `_root`. If the root node is a parent node, the left child object and right child object can be accessed by using the syntax `_root.children[0]` and `_root.children[1]`. Since the dictionary expects an array of values, the tree weights like `features` and `thresholds` must be fetched from each node object, one at a time.

Below describes the syntax to retrieve the tree weights from each node in a Hoeffding decision tree classifier.

For `children_left`, `children_right`, `children_default`, `features` and `thresholds`, please refer to the same subsection in the Regression section since the syntax are the same. 

1. values: Both parent nodes and leaf nodes contain an attribute called `stats`. The `stats` attribute references a Python dictionary that stores the observed class weights. The prediction value is retrieved by using the syntax `node.stats`. 
    - The prediction value is calculcated differently for random forest classifier and adaptive random forest classifier. Regardless, both models use the prediction value to calculate the prediction probabilities by normalizing over the sum of the weights. 
    - Random forest classifier: When building the tree, after a new node is created, the prediction value is obtained from the node splitter as shown in the <a href="https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/tree/_tree.pyx#L496">source code</a>. In the case of binary classification, the prediction value contains the weighted sample counts of the left split and right split.
    - Adaptive random forest classifier: The class weight is updated each time a new sample reaches at the current leaf node. The weight value that updates the `stats` is not constant since it is drawn from a Poisson distribution for every incremental training, as shown in the <a href="https://github.com/online-ml/river/blob/main/river/ensemble/adaptive_random_forest.py#L73">source code</a>.
    - However, it was later proved that the `node.stats` cannot be used to calculate accurate SHAP values when continously training on new samples. The details can be found at `FYP2_Lead_Scoring_Explainer.ipynb`.


2. node_sample_weight: Both parent nodes and leaf nodes contain an attribute called `stats`. The `stats` attribute references a Python dictionary that stores the observed class weights. Like Scikit-learn's decision tree, the weighted count of samples at the current node is calculated as the sum of the weights in `stats`. However, it was later proved that the sum of the weights in `stats` cannot be used to calculate accurate SHAP values when continously training on new samples. The details can be found at `FYP2_Lead_Scoring_Explainer.ipynb`.

### Validation using Tree SHAP Explainer

A SHAP tree explainer is initialized with the imported dictionary to check that the extraction function is successful. 

**Reminder**: The River model is not loaded here since the River library is incompatible with SHAP library. Instead, the dictionary containing the extracted tree weights is imported here in JSON format. 

Load the dictionary containing the extracted tree weights.

In [31]:
with open('outputs/lead_scoring/arf_cf.json', 'r') as f:
    ls_arf_dict_serializable = json.load(f)

ls_arf_dict = deserialize_arf(ls_arf_dict_serializable)

Load test data to compute SHAP value and serve as background dataset for SHAP tree loss explainer. Randomly choose 300 samples from the test set to serve as the background dataset.

In [32]:
# Load data preprocessor
with open('outputs/lead_scoring/data_preprocessor.pkl', 'rb') as f:
    ls_data_pp = pickle.load(f)

# Load car test dataset
ls_test = pd.read_csv(f'outputs/lead_scoring/test_set.csv')

# Preprocess X test
ls_X_test = ls_test.drop(columns=['converted', 'dont_call'], axis=1)
ls_X_test = ls_data_pp.preprocess(ls_X_test)
ls_y_test = ls_test['converted']

print(f'X_test shape: {ls_X_test.shape}')
print(f'y_test shape: {ls_y_test.shape}')

# Choose a subset of the ls_X_test
rng = np.random.default_rng(2022)
idx_arr = rng.choice(range(len(ls_X_test)), 300)
ls_X_test_subsample = ls_X_test.iloc[idx_arr, :].copy()

X_test shape: (1305, 9)
y_test shape: (1305,)


Initialize the SHAP tree explainer using the deserialized dictionary and calculate the SHAP value. The details of the explainer is discussed in *FYP2_Lead_Scoring_Explainer.ipynb*. In this case, the SHAP tree explainer is using `tree_path_dependent` approach.

In [33]:
# Compute SHAP value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(ls_arf_dict)}')
for _ in range(num_iter):
    ls_tree_explainer = shap.TreeExplainer(
        model = copy.deepcopy(ls_arf_dict), 
        feature_perturbation = 'tree_path_dependent'
    )
    _ = ls_tree_explainer.shap_values(ls_X_test)

end = time.time()
print(f'Average time taken to calculate SHAP value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'dict'>
Average time taken to calculate SHAP value: 0.1352790117263794 seconds


The SHAP tree explainer is also initialized using Scikit-learn random forest classifier and SHAP value is calculated. This is to check that the performance of SHAP value calculation is not affected when dictionary is used. The result shows that the performance of SHAP value calculation is similar to the previous performance. In this case, the SHAP tree explainer is using `tree_path_dependent` approach.

In [34]:
# Compute SHAP value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(ls_trf_model)}')
for _ in range(num_iter):
    ls_tree_explainer_tmp = shap.TreeExplainer(
        model = ls_trf_model, 
        feature_perturbation = 'tree_path_dependent'
    )
    _ = ls_tree_explainer_tmp.shap_values(ls_X_test)

end = time.time()
print(f'Average time taken to calculate SHAP value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
Average time taken to calculate SHAP value: 0.14020864963531493 seconds


Initialize the SHAP tree explainer using the deserialized dictionary and calculate the SHAP value. The details of the explainer is discussed in *FYP2_Lead_Scoring_Explainer.ipynb*. In this case, the SHAP tree explainer is using `interventional` approach. It can be observed that SHAP tree explainer using `interventional` approach is slower than the one using `tree_path_dependent` approach.

In [35]:
# Compute SHAP loss value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(ls_arf_dict)}')
for _ in range(num_iter):
    ls_tree_loss_explainer = shap.TreeExplainer(
        model = copy.deepcopy(ls_arf_dict), 
        feature_perturbation = 'interventional', 
        data = ls_X_test_subsample
    )
    _ = ls_tree_loss_explainer.shap_values(ls_X_test)

end = time.time()
print(f'Average time taken to calculate SHAP value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'dict'>
Average time taken to calculate SHAP value: 1.5576391458511352 seconds


The SHAP tree explainer is also initialized using Scikit-learn random forest classifier and SHAP value is calculated. This is to check that the performance of SHAP value calculation is not affected when dictionary is used. The result shows that the performance of SHAP value calculation is similar to the previous performance. In this case, the SHAP tree explainer is using `interventional` approach. It can be observed that SHAP tree explainer using `interventional` approach is slower than the one using `tree_path_dependent` approach.

In [36]:
# Compute SHAP loss value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(ls_arf_dict)}')
for _ in range(num_iter):
    ls_tree_loss_explainer = shap.TreeExplainer(
        model = copy.deepcopy(ls_arf_dict), 
        feature_perturbation = 'interventional', 
        data = ls_X_test_subsample
    )
    _ = ls_tree_loss_explainer.shap_values(ls_X_test)

end = time.time()
print(f'Average time taken to calculate SHAP value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'dict'>
Average time taken to calculate SHAP value: 1.4959629774093628 seconds


### Validation using Tree SHAP Loss Explainer

A SHAP tree loss explainer is initialized with the imported dictionary to check that the extraction function is successful.

Initialize the SHAP tree loss explainer using the deserialized dictionary and calculate the SHAP loss value. The details of the explainer is discussed in *FYP2_Lead_Scoring_Explainer.ipynb*.

In [37]:
# Compute SHAP loss value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(ls_arf_dict)}')
for _ in range(num_iter):
    ls_tree_loss_explainer = shap.TreeExplainer(
        model = copy.deepcopy(ls_arf_dict), 
        feature_perturbation = 'interventional', 
        data = ls_X_test_subsample, 
        model_output = 'log_loss'
    )
    _ = ls_tree_loss_explainer.shap_values(ls_X_test, ls_y_test)

end = time.time()
print(f'Average time taken to calculate SHAP loss value: {(end - start)/num_iter} seconds')

Model used in initialization: <class 'dict'>
Average time taken to calculate SHAP loss value: 1.8151605606079102 seconds


The SHAP tree loss explainer is also initialized using Scikit-learn random forest classifier and SHAP loss value is calculated. This is to ensure that the performance of SHAP loss value calculation is not affected when dictionary is used. The result below shows that the performance of SHAP loss value calculation is similar to previous performance.

In [38]:
# Compute SHAP loss value
start = time.time()
num_iter = 10

print(f'Model used in initialization: {type(ls_trf_model)}')
for _ in range(num_iter):
    ls_tree_loss_explainer_tmp = shap.TreeExplainer(
        model = ls_trf_model, 
        feature_perturbation = 'interventional', 
        data = ls_X_test_subsample, 
        model_output = 'log_loss'
    )
    _ = ls_tree_loss_explainer_tmp.shap_values(ls_X_test, ls_y_test)

end = time.time()
print(f'Average time taken to calculate SHAP loss value : {(end - start)/num_iter} seconds')

Model used in initialization: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
Average time taken to calculate SHAP loss value : 1.8570066690444946 seconds


### Validation using Dictionary

As shown in *FYP2_Lead_Scoring_Model_Training.ipynb*, `cp_trf_model` is the TRF model that transfer its tree structure to the ARF model where its weights are extracted to `cp_arf_dict`. In that transfer learning process, it is validated that the tree structure is exactly the same. The tree structure is the same such that for every parent node *i*, the `split feature`, `split threshold`, `child nodes`, `prediction_value`, and `node_sample_weight` are the same for both models. The `prediction value` and `node_sample_weight` are the same since the prediction value of every node is directly copied from the TRF model to the ARF model. It is technically feasible since both models use majority class prediction as the leaf prediction mechanisms to output prediction probabilities. The `node_sample_weight` is also the same since it is calculcated as the sum of weights in `prediction_value`.

Given the high similarity in tree structures, the `cp_trf_model`'s weights are extracted to a dictionary named `cp_trf_dict` to compare with `cp_arf_dict`. The `cp_trf_dict` is used to initialize a SHAP tree explainer to verify its correctness.

In [39]:
ls_trf_dict = {
    'internal_dtype' : ls_trf_model.estimators_[0].tree_.value.dtype.type,
    'input_dtype'    : np.float32,
    'objective'      : 'binary_crossentropy',
    'tree_output'    : 'probability',
    'base_offset'    : 0,
    'trees'          : []
}

for dt in ls_trf_model.estimators_:
    dt = dt.tree_
    
    singletree = {
        'children_left'     : dt.children_left,
        'children_right'    : dt.children_right,
        'children_default'  : dt.children_left,
        'features'          : dt.feature,
        'thresholds'        : dt.threshold,
        'node_sample_weight': dt.weighted_n_node_samples
    }

    values = dt.value.reshape(
        dt.value.shape[0], 
        dt.value.shape[1] * dt.value.shape[2]
    )
    # Normalize the prediction value using sum for each node
    values = (values.T / values.sum(1)).T
    # Divide by total number of base learners
    scaling = 1 / len(ls_trf_model.estimators_)
    values = values * scaling
        
    singletree['values'] = values
    
    ls_trf_dict['trees'].append(singletree)
    
# Compute SHAP loss value
ls_tree_loss_explainer_tmp2 = shap.TreeExplainer(
    model = copy.deepcopy(ls_trf_dict), 
    feature_perturbation = 'interventional', 
    data = ls_X_test_subsample
)
_ = ls_tree_loss_explainer_tmp2.shap_values(ls_X_test, ls_y_test)

**Test 1**: The field values `internal_dtype`, `objective`, `tree_output`, `base_offset` for both dictionaries are the same. As mentioned earlier, for the Scikit-learn-implemented tree models, the `input_dtype` value must be set to `np.float32` to get the exact matches to predictions.

In [40]:
print('Scikit-learn random forest classifier:\n')
print(f'\tinternal_dtype: {ls_trf_dict["internal_dtype"]}')
print(f'\tinput_dtype   : {ls_trf_dict["input_dtype"]}')
print(f'\tobjective     : {ls_trf_dict["objective"]}')
print(f'\ttree_output   : {ls_trf_dict["tree_output"]}')
print(f'\tbase_offset   : {ls_trf_dict["base_offset"]}')

print('\nRiver adaptive random forest classifier:\n')
print(f'\tinternal_dtype: {ls_arf_dict["internal_dtype"]}')
print(f'\tinput_dtype   : {ls_arf_dict["input_dtype"]}')
print(f'\tobjective     : {ls_arf_dict["objective"]}')
print(f'\ttree_output   : {ls_arf_dict["tree_output"]}')
print(f'\tbase_offset   : {ls_arf_dict["base_offset"]}')

Scikit-learn random forest classifier:

	internal_dtype: <class 'numpy.float64'>
	input_dtype   : <class 'numpy.float32'>
	objective     : binary_crossentropy
	tree_output   : probability
	base_offset   : 0

River adaptive random forest classifier:

	internal_dtype: <class 'numpy.float64'>
	input_dtype   : <class 'numpy.float64'>
	objective     : binary_crossentropy
	tree_output   : probability
	base_offset   : 0


**Test 2**: The position of all the nodes must be the same. It must be ensured that for every *i*th node, the split feature and split threshold must be the same between the two dictionaries. Note that the node's array index position for *i*th node is often not the same between the two dictionaries. Hence, the node index for decision tree and Hoeffding tree must be tracked separately to access the respective arrays of tree weights.

In [41]:
df = pd.DataFrame([], columns=['base_learner_no', 'dt_node_idx', 'ht_node_idx', 
                               'same_feature?', 'same_threshold?'])

node_id = -1

for base_learner_no in range(len(ls_trf_model.estimators_)):
    queue = []
    # Separately track the node index position for both dictionaries
    queue.append((0, 0))

    while len(queue) > 0:
        dt_node_idx, ht_node_idx = queue.pop(0)
        dt_node_idx = int(dt_node_idx)
        ht_node_idx = int(ht_node_idx)

        # Retrieve the features from the dictionaries
        ht_node_feature = ls_arf_dict["trees"][base_learner_no]["features"][ht_node_idx]
        dt_node_feature = ls_trf_dict["trees"][base_learner_no]["features"][dt_node_idx]

        # Retrieve the threshold from the dictionaries
        ht_node_threshold = ls_arf_dict["trees"][base_learner_no]["thresholds"][ht_node_idx]
        dt_node_threshold = ls_trf_dict["trees"][base_learner_no]["thresholds"][dt_node_idx]

        # Add records to dataframe for debugging
        node_id += 1
        df.loc[node_id, :] = [base_learner_no+1, dt_node_idx, ht_node_idx, 
                              ht_node_feature == dt_node_feature, 
                              ht_node_threshold == dt_node_threshold]

        # Retrieve the left child index position from the dictionaries
        ht_left_ch_idx = ls_arf_dict["trees"][base_learner_no]["children_left"][ht_node_idx]
        dt_left_ch_idx = ls_trf_dict["trees"][base_learner_no]["children_left"][dt_node_idx]

        # Retrieve the right child index position from the dictionaries
        ht_right_ch_idx = ls_arf_dict["trees"][base_learner_no]["children_right"][ht_node_idx]
        dt_right_ch_idx = ls_trf_dict["trees"][base_learner_no]["children_right"][dt_node_idx]

        # Only enqueue if the child node is a branch node
        if dt_left_ch_idx != -1:
            queue.append((dt_left_ch_idx, ht_left_ch_idx))
        if dt_right_ch_idx != -1:
            queue.append((dt_right_ch_idx, ht_right_ch_idx))

df.head(10)

,base_learner_no,dt_node_idx,ht_node_idx,same_feature?,same_threshold?
0,1,0,0,True,True
1,1,1,1,True,True
2,1,24,2,True,True
3,1,2,3,True,True
4,1,11,4,True,True
5,1,25,5,True,True
6,1,32,6,True,True
7,1,3,7,True,True
8,1,8,8,True,True
9,1,12,9,True,True


The number of problematic nodes should be zero as shown below.

In [42]:
problematic_nodes = df[(~df['same_feature?']) | (~df['same_threshold?'])]
print(f'Number of problematic nodes: {problematic_nodes.shape[0]}')
problematic_nodes

Number of problematic nodes: 0


,base_learner_no,dt_node_idx,ht_node_idx,same_feature?,same_threshold?


**Test 3**: If test 2 is successful, check the average weights difference between Scikit-learn RF classifier and River ARF classifier. As shown below, the average weights difference for `features`,  `thresholds`, `values`, and `node_sample_weight` must be zero since the test 2 has passed. 

In [43]:
weight_types = ['features', 'thresholds', 'values', 'node_sample_weight']

df = pd.DataFrame([], columns=weight_types)

for weight_type in weight_types:
    for idx in range(len(ls_trf_model.estimators_)):
        dt_weights = np.sort(ls_arf_dict["trees"][idx][weight_type].flatten())
        ht_weights = np.sort(ls_trf_dict["trees"][idx][weight_type].flatten())
        diffs = ht_weights - dt_weights
        diffs = diffs[diffs != 0]
        diffs_mean = np.abs(diffs).mean() if diffs.shape[0] > 0 else 0
        df.loc[idx, f'{weight_type}'] = diffs_mean

print('Average weights difference between Scikit-learn RF classifier and River ARF classifier:')
display(df)

Average weights difference between Scikit-learn RF classifier and River ARF classifier:


,features,thresholds,values,node_sample_weight
0,0,0,0.0,0
1,0,0,0.0,0
2,0,0,0.0,0
3,0,0,0.0,0
4,0,0,0.0,0
5,0,0,0.0,0
6,0,0,0.0,0
7,0,0,0.0,0
8,0,0,0.0,0
9,0,0,0.0,0


The end.